Необходимо решить задачу заказчика на анализ тональности отзывов на товары при наличии тестовой выборки в размере 100 отзывов. Требуется качество больше 85%. Ссылка на соревнование Kaggle: https://inclass.kaggle.com/c/product-reviews-sentiment-analysis

В этом ноутбуке реализован парсер сбора обучающей выборки с разметкой, прдобработка данных, и обучение модели.

## Как выглядит тестовая выборка

In [2]:
import pandas as pd

sample = pd.read_csv('sample_submission.csv', index_col='Id', sep=',')
sample[:5]

,y
Id,
0,neg
1,neg
2,neg
3,neg
4,neg


In [3]:
import codecs
import re

test = codecs.open('test.csv', 'r', 'utf_8_sig').read()
test = re.sub(r'[^\w]', ' ', test).lower()    # чистим от знаков
test = test.split('review')[1::2]   # разделяем на список
test[:2]

[' ужасно слабый аккумулятор  это основной минус этого аппарата  разряжается буквально за пару часов при включенном wifi и на макс подсветке  например если играть или смотреть видео  следовательно использовать можно только если есть постоянная возможность подзарядиться  качества звука через динамик далеко не на высоте наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню  очень мало встроенной памяти  а приложения устанавливаются именно туда  с этим связанны неудобства   нужно постоянно переносить их на карту памяти  несколько неудобно что нету отдельной кнопки для фото  подумываю купить батарею большей емкость мб что нибудь измениться    ',
 ' ценанадежность неубиваемостьдолго держит батарею 4 дня стабильно как телефон  3 4 как плеер если  постоянно долбиться в уши и звонить по паре часо на дню  игры и  конечно   смс   в месяц около 200 шт набирается   максимальное время работы 5 дней в щадящем режиме 2 simqwerty рулит  после нее набор смс на обычных сенсорниках и

## Собираем обучающую выборку

In [ ]:
import requests
import bs4

headers = {'accept': '*/*',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
links = open('links.txt').read().split('\n')

# получаем ссылки на бренды (группы) телефонов
session = requests.Session()
req = session.get('https://slonrekomenduet.com/category/smartphones.html', headers=headers)
parser = bs4.BeautifulSoup(req.text, 'lxml')
raw_links = parser.findAll('li')[:80]
links = []
for link in raw_links:
    link = re.search('/category/.+\.html', str(link.a))
    links.append(link.group(0))

# получаем ссылки на модели телефонов
model_links = []
for link in links:
    session = requests.Session()
    req = session.get('https://slonrekomenduet.com{}'.format(link), headers=headers)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    raw_links = parser.findAll('div', {'class': 'model'})[:-10]
    for model_link in raw_links:
        model_links.append(re.search('/model/.+\.html', str(model_link.a)).group(0))

# парсим страницы с отзывами
reviews = []
ratings = []
for i, link in enumerate(model_links):
    print(i, round(i/1567, 4), link)    # наблюдаем ход парсинга
    session = requests.Session()
    req = session.get('https://slonrekomenduet.com{}'.format(link), headers=headers)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    try:
        n_pages = int(re.findall('page/\d', str(parser))[-1][-1])
    except IndexError:
        n_pages = 1
    for page in range(1, n_pages+1):
        review = parser.findAll('div', {'class': 'comment_text'})
        for rev in review:
            text = rev.text.lower()    # приводим к нижнему регистру
            text = text.replace('\xa0', ' ')
            text = re.sub(r'[^\w]', ' ', text)    # избавляемся от знаков препинания
            reviews.append(text)
        rating = parser.findAll('div', {'class': 'br-widget'})[1:len(review)+1]
        for rate in rating:
            ratings.append(len(rate.findAll('a', {'class': 'br-active'})))

df = pd.DataFrame({'review': reviews, 'rating': ratings})
df.to_csv('reviews.csv')

In [ ]:
# Открываем обучающую выборку из сохраненного файла
df = pd.read_csv('reviews.csv').dropna()

## Пример отзыва и (не)сбалансированность классов

In [5]:
print(df['review'][0])
df['rating'].value_counts()    # смотрим на сбалансированность классов

достоинства   удобно лежит в детской руке  неплохой дизайн  отличная задумка    недостатки но реализация отвратительная  глюк на глюке   за два месяца использования столкнулись со следующими проблемами  спервого дня использования   1  не видит симку  приходится разбирать и колдовать  поворачивая ее  по полмиллиметра в разные стороны  да  слот под симку чуть больше симки   2  провод приходится перетыкать в гнездо по несколько раз  чтобы пошла зарядка   3  родительский контроль работает через раз  в частности контроль приложений чудачит как хочет  то появляется значок блокировки приложения  когда оно доступно  то наоборот  4  настроить отслеживание местоположения так и не удалось  несмотря на два дня танцев с бубнами  ни мой телефон  nexus 5x   ни мужа  iphone 4s   так и не смогли подключиться к устройству  при помощи сканирования кода через официальное приложение  пишет  ошибка подключения   и хоть тресни  5  к wi fi ни разу нормально не подключился  все время вылетает  комментарий ну и

5    40740
4    16654
3     9707
1     8279
2     6785
Name: rating, dtype: int64

## Разметка на бинарные классы
Отзывы с оценками 4 и 5 - положительные, остальные отрицательные.

In [11]:
df['label'] = ['pos' if x > 3 else 'neg' for x in df['rating']]
df['label'].value_counts()

pos    57394
neg    24771
Name: label, dtype: int64

## Векторизация по частотности и обучение логистической регрессии

Настраиваемые параметры: количество слов от 1 до 3, стоп-слова исключаются.


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))

ppl = Pipeline([
    ('vectorizer', CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))),
    ('classifier', LogisticRegression(solver='liblinear')),
])

print('Качество на обучении:', cross_val_score(ppl, df['review'], df['label'], scoring='accuracy', cv=5).mean())

Качество на обучении: 0.8509342276361581


## Делаем предсказание на тестовой выборке

In [13]:
ppl.fit(df['review'], df['label'])
pred = ppl.predict(test)
pred = pd.DataFrame({'Id': list(range(100)), 'y': pred})
pred.to_csv('pred.csv', sep=',', index=False)

In [14]:
ppl.predict(test)

array(['neg', 'neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'pos', 'neg',
       'pos', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos',
       'pos', 'pos', 'pos', 'neg', 'pos', 'pos', 'pos', 'neg', 'pos',
       'pos', 'neg', 'pos', 'neg', 'pos', 'neg', 'pos', 'neg', 'pos',
       'pos', 'pos', 'pos', 'neg', 'pos', 'pos', 'pos', 'neg', 'neg',
       'pos', 'pos', 'pos', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg',
       'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'neg', 'pos', 'neg',
       'neg', 'neg', 'neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'neg',
       'neg', 'neg', 'neg', 'pos', 'neg', 'neg', 'neg', 'neg', 'neg',
       'pos', 'neg', 'neg', 'pos', 'neg', 'pos', 'pos', 'neg', 'neg',
       'pos', 'pos', 'neg', 'pos', 'neg', 'neg', 'pos', 'neg', 'pos',
       'pos'], dtype=object)